In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob

In [ ]:
#data_dir = "G:\\1_Data1\\93_BaikalJun2023Sea\\meteo\\"
#data_dir = "G:\\1_Data1\\90_BaikalJun2022Sea\\Meteo\\"
#data_dir = "G:\\1_Data1\\98_BaikalJul2024Sea\\origin\\"
#data_dir = r'c:\Users\redmi\1_Data1\sea\meteo'
data_dir = r'G:\\1_Data1\\sea\\meteo'
pwd = data_dir
data_files = glob.glob(pwd + r"\*.txt")
data_files

In [ ]:
f=data_files[1]
df = pd.read_csv(f,
                 encoding_errors='ignore',
                 header=None,
                 skiprows=[0],
                 engine='python',
                 skipfooter=1,
                 sep=' ',
                 names=['date', 'time', 'Temp', 'Wh', 'Wdir', 'vx', 'vy', 'vz', 'Press', 'RH'],
                 #parse_dates={'DateTime':['date', 'time']},
                 #dayfirst=True,
                 on_bad_lines='skip',
                )
df['DateTime'] = pd.to_datetime(df['date'] + ' ' + df['time'], dayfirst=True, cache=True)

df.resample('1min', on='DateTime').agg({'vx':'mean', 'vy':'mean', 'Temp':'mean', 'RH': 'mean', 'Press': 'mean', 'Wh':lambda x:x, 'Wdir':lambda x:x})

In [ ]:
dfl = []
for f in data_files:
    df = pd.read_csv(f,
                     encoding_errors='ignore',
                     header=None,
                     skiprows=[0],
                     engine='python',
                     skipfooter=1,
                     sep=' ',
                     names=['date', 'time', 'Temp', 'Wh', 'Wdir', 'vx', 'vy', 'vz', 'Press', 'RH'],
                     #parse_dates={'DateTime':['date', 'time']},
                     #dayfirst=True,
                     on_bad_lines='skip',
                    )
    df['DateTime'] = pd.to_datetime(df['date'] + ' ' + df['time'], dayfirst=True, cache=True)
    df = df.resample('1min', on='DateTime').agg({'vx':'mean', 'vy':'mean', 'Temp':'mean', 'RH': 'mean', 'Press': 'mean', 'Wh':lambda x:x, 'Wdir':lambda x:x})
    
    dfl.append(df)
    del(df)
df = pd.concat(dfl)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.sort_values(by='DateTime', inplace=True)
df

In [ ]:
def filter_df(df):
    pass

In [ ]:
df.columns

In [ ]:
df.plot(x=df.index, y=['Temp', 'RH'], subplots=True)

In [ ]:
df.plot(x='DateTime', y=['Wh', 'Wdir'], subplots=True)

In [ ]:
df.to_csv(pwd+'//..//meteo.txt', sep='\t')

In [ ]:
plt.rcParams['figure.figsize']=[15, 5]
plt.scatter('DateTime', 'Wh', data=df, s=1, alpha=0.1)
plt.xlim(pd.to_datetime('2021-06-01 00:00'), pd.to_datetime('2021-06-15 00:00'))
plt.legend()

In [ ]:
dfm=df.copy()

In [ ]:
df = pd.read_csv('G:/1_Data1/sea/AllSea.txt',
                 header=0,
                 # skip_blank_lines=True,
                 skipinitialspace=True,
                 #na_values='--',
                 sep='\t',
                 decimal='.',
                 # nrows=100000,
                 parse_dates=['DateTime'],
                 dayfirst=False
                )


In [ ]:
colstoleft = ['DateTime', 'Longitude', 'Latitude', 'Speed', 'Course']

In [ ]:
df.drop([i for i in df.columns if i not in colstoleft],axis='columns', errors='ignore', inplace=True)

In [ ]:
df.info()

In [ ]:
df = pd.merge(left=df, right=dfm, left_on='DateTime', right_on='DateTime', how='outer')

In [ ]:
df.sort_values(by='DateTime', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.dropna(subset=['DateTime'], axis='index')


In [ ]:
len(set(df['DateTime']))

In [ ]:
df['Lat'] = df['Latitude']#.rolling(3, min_periods=1, center=True).mean()
df['Long'] = df['Longitude']#.rolling(3, min_periods=1, center=True).mean()

In [ ]:
df.index = df['DateTime']
df

In [ ]:
df['Lat'] = df['Lat'].interpolate(method='time')
df['Long'] = df['Long'].interpolate(method='time')

In [ ]:
df.reset_index(drop=True)

In [ ]:
df['mywindspeed'] = np.sqrt(np.power(df['vx'],2)+np.power(df['vy'],2))

In [ ]:
df['mywinddir'] = np.arctan2(-df['vy'], df['vx']) * 180 / np.pi + 180

In [ ]:
plt.rcParams['figure.figsize']=[15, 5]
plt.scatter('DateTime', 'mywinddir', data=df, s=1, c='red')
plt.scatter('DateTime', 'Wdir', data=df, s=1, alpha=0.1)
plt.xlim(pd.to_datetime('2021-06-01 00:00'), pd.to_datetime('2021-06-15 00:00'))

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64')/10**9
df['dt'] = df['DateSec'].shift(-1) - df['DateSec'].shift(1)

In [ ]:
R_EARTH = 6371000  #meters
df['vesselvy'] = (df['Lat'].shift(-1) - df['Lat'].shift(1)) * 2 * np.pi * R_EARTH / 360
df['vesselvx'] = (df['Long'].shift(-1) - df['Long'].shift(1)) * (np.cos(df['Lat'] * np.pi / 180 ) * 2 * np.pi * R_EARTH/360)
df['vesselspeed'] = np.sqrt(np.power(df['vesselvy'],2) + np.power(df['vesselvx'],2))/df['dt']
df.describe()

In [ ]:
plt.scatter('DateTime', 'Speed', data=df, c='red', s=20, alpha=0.5)
plt.scatter('DateTime', 'vesselspeed', data=df, c='blue', s=5)
plt.xlim(pd.to_datetime('2023-06-03 12:00'), pd.to_datetime('2023-06-04 0:00'))
plt.xlim(pd.to_datetime('2023-06-03 17:45'), pd.to_datetime('2023-06-03 17:59'))

plt.ylim(0, 8)
plt.grid()

In [ ]:
plt.scatter('DateTime', 'Latitude', data=df, c='red', s=15, alpha=0.4)
plt.scatter('DateTime', 'Lat', data=df, c='blue', s=3, alpha=1)
plt.xlim(pd.to_datetime('2023-06-03 17:25'), pd.to_datetime('2023-06-03 17:35'))
plt.xlim(pd.to_datetime('2023-06-03 17:45'), pd.to_datetime('2023-06-03 17:59'))

plt.ylim(52.43, 52.46)
plt.grid()

In [ ]:
df.loc[(df['DateTime'] > pd.to_datetime('2023-06-03 17:45')) & (df['DateTime'] < pd.to_datetime('2023-06-03 17:59')), ['DateTime', 'Longitude', 'Latitude', 'Long', 'Lat', 'vesselspeed', 'dt', 'vx']]

In [ ]:
df.loc[(df['DateTime'] > pd.to_datetime('2023-06-03 17:30')) & (df['DateTime'] < pd.to_datetime('2023-06-03 17:35')), ['DateTime', 'vx', 'vy', 'Longitude', 'Latitude', 'Long', 'Lat', 'dt']]

In [ ]:
plt.scatter('DateTime', 'Longitude', data=df, c='red', s=15, alpha=0.4)
plt.scatter('DateTime', 'Long', data=df, c='blue', s=3, alpha=1)
plt.xlim(pd.to_datetime('2023-06-03 17:30'), pd.to_datetime('2023-06-03 17:35'))
plt.ylim(105.95, 106.05)
plt.grid()

In [ ]:
df.describe()

In [ ]:
df.interpolate(method ='linear', limit_direction ='forward', limit_area='inside').describe()

In [ ]:
df.describe()